In [1]:
#JSONとOAuth認証用のライブラリを使用
import json
from requests_oauthlib import OAuth1Session
import setting
import tweepy
import os

twitter = OAuth1Session(setting.CONSUMER_KEY, setting.CONSUMER_SECRET, setting.ACCESS_TOKEN, setting.ACCESS_SECRET)

In [2]:
# !pip install tweepy

In [3]:
keyword = "#バンキシャ"
q = keyword
#認証
auth = tweepy.OAuthHandler( setting.CONSUMER_KEY, setting.CONSUMER_SECRET )
auth.set_access_token( setting.ACCESS_TOKEN, setting.ACCESS_SECRET)

api = tweepy.API(auth, wait_on_rate_limit = True)

#検索キーワード設定 
q = keyword

#つぶやきを格納するリスト
tweets_data =[]

#カーソルを使用してデータ取得
for tweet in tweepy.Cursor(api.search, q=q, count=100,tweet_mode='extended').items():

    #つぶやきテキスト(FULL)を取得
    tweets_data.append(tweet.full_text + '\n')

#出力ファイル名
dfile = "../data/hash_tag"
fname = "init.txt"
if not os.path.exists( dfile):
    os.mkdir(dfile)

#ファイル出力
with open(dfile+"/"+fname, "w",encoding="utf-8") as f:
    f.writelines(tweets_data)

# print
for i in range(len(tweets_data)):
    if i % 100 == 0: 
        print(tweets_data[i])

RT @bankisha: 新コーナー『ベタバリ』の第1回放送が
YouTubeにアップされました！
見逃した方は是非ご覧ください

桝キャスターが取材 コロナ対策"司令塔"の一日『ベタバリ』
（2021年4月4日放送「真相報道バンキシャ！」より） 
https://t.co/…

RT @nittele_ntv: ＼毎日当たる／
#BangBang新番特番 キャンペーン✨

今夜6時からの『#バンキシャ』の実況や感想を投稿して【プライベート・ビエラ】を当てよう🎁
①@nittele_ntvをフォロー
②↓のボタンからツイート

フォロー＆RTキャンペーン…

RT @bankisha: 【バン犬と1分だけトーク】
放送後のとっておきトーク！
#古市憲寿 さんが 初登場 #伊沢拓司 さんに質問タイム！？
@poe1985 @tax_i_ 
#バンキシャ https://t.co/e9OGci44OJ

RT @bankisha: 【バン犬と1分だけトーク】
放送後のとっておきトーク！
#古市憲寿 さんが 初登場 #伊沢拓司 さんに質問タイム！？
@poe1985 @tax_i_ 
#バンキシャ https://t.co/e9OGci44OJ

【番組について一言】 #バンキシャ #BangBang新番特番 @nittele_ntv

RT @TONKO216: 夏目さん本人の結婚報告
#夏目三久 #バンキシャ https://t.co/miBb0YsuKR

【おめでとう】 #バンキシャ #BangBang新番特番 @nittele_ntv

RT @takechan7th: 生まれ故郷大阪。
道頓堀の屋台舟でマスクをせずにさわぐ若者達。。。

#バンキシャ
#コロナ自粛

気の緩みや、ストレスもそらあるやろ。ただ、人間はサルとちゃうねん。

世の中を俯瞰して、深掘りして、考えて、振り返って。

これができるのが、人…

【番組について一言】 #バンキシャ #BangBang新番特番 @nittele_ntv

【いいよね】 #バンキシャ #BangBang新番特番 @nittele_ntv

RT @nittele_ntv: ＼毎日当たる／
#BangBang新番特番 キャンペーン✨

今夜6時からの『#バンキシャ』の実況や感想を投稿して【プライベート・ビエラ】を

In [4]:
# pip install bert

In [5]:
# pip install transformers

In [6]:
import torch
from torch import nn
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import numpy as np
model = BertModel.from_pretrained('bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers')
tokenizer_bert  = BertTokenizer("bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt",
                               do_lower_case=False, do_basic_tokenize=False)

In [9]:
import re
import string
import torchtext
# from bert import BertTokenizer
# from bert import BertTokenizer

def preprocessing_text(text):
    text = re.sub("<br/>", "", text)
    
    for p in string.punctuation:
        if (p==".")or (p==","):
            continue
        else:
            text=text.replace(p, "")
            
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")

    return text

def tokenizer_with_preprocessing(text, tokenizer=tokenizer_bert.tokenize):
    text = preprocessing_text(text)
    ret = tokenizer(text)
    return ret

max_length = 256

TEXT = torchtext.data.Field(
    sequential = True, tokenize=tokenizer_with_preprocessing,
    use_vocab = True, lower = True, include_lengths = True,
    batch_first = True, fix_length= max_length,
    init_token="[CLS]",eos_token = "[SEP]",pad_token="[PAD]",
    unk_token=["UNK"])





In [ ]:
class BertForIMDb(nn.Module):
    '''BERTモデルにIMDbのポジ・ネガを判定する部分をつなげたモデル'''

    def __init__(self, net_bert):
        super(BertForIMDb, self).__init__()

        # BERTモジュール
        self.bert = net_bert  # BERTモデル

        # headにポジネガ予測を追加
        # 入力はBERTの出力特徴量の次元、出力はポジ・ネガの2つ
        self.cls = nn.Linear(in_features=768, out_features=2)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=False, attention_show_flg=False):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        token_type_ids： [batch_size, sequence_length]の、各単語が1文目なのか、2文目なのかを示すid
        attention_mask：Transformerのマスクと同じ働きのマスキングです
        output_all_encoded_layers：最終出力に12段のTransformerの全部をリストで返すか、最後だけかを指定
        attention_show_flg：Self-Attentionの重みを返すかのフラグ
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        if attention_show_flg == True:
            '''attention_showのときは、attention_probsもリターンする'''
            encoded_layers, pooled_output, attention_probs = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)
        elif attention_show_flg == False:
            encoded_layers, pooled_output = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)

        # 入力文章の1単語目[CLS]の特徴量を使用して、ポジ・ネガを分類します
        vec_0 = encoded_layers[:, 0, :]
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_sizeに変換
        out = self.cls(vec_0)

        # attention_showのときは、attention_probs（1番最後の）もリターンする
        if attention_show_flg == True:
            return out, attention_probs
        elif attention_show_flg == False:
            return out